In [ ]:
import pymongo
import pandas as pd
from tqdm import tqdm

from notebook_modules.database import Database
from notebook_modules.half import Half
from notebook_modules.distribution import aggregate, distribute
from notebook_modules.lists import make_list
from notebook_modules.plots import make_distribution_plot

In [ ]:
db = Database()
assert db.client, "No database client available!"
stackoverflow = db.client["stackoverflow"]
tags = stackoverflow["tags"]
posts = stackoverflow["posts"]

In [ ]:
tags_h1 = aggregate(posts, Half.make_half(2019, 1))
tags_h2 = aggregate(posts, Half.make_half(2019, 2))

In [ ]:
top_h1 = tags_h1.head(int(0.2 * len(tags_h1)))
top_h2 = tags_h2.head(int(0.2 * len(tags_h2)))
bottom_h1 = tags_h1.iloc[len(top_h1):]
bottom_h2 = tags_h2.iloc[len(top_h2):]
assert len(tags_h1) == len(top_h1) + len(bottom_h1), "Split dataframe length mismatch!"
assert len(tags_h2) == len(top_h2) + len(bottom_h2), "Split dataframe length mismatch!"

In [ ]:
undecaying = pd.merge(top_h1, top_h2, how="inner", on="tag")
decaying_left_join = pd.merge(top_h1, top_h2, how="left", on="tag", indicator=True)
decaying = decaying_left_join[decaying_left_join["_merge"] == "left_only"].reset_index(drop=True)
decaying.drop("_merge", axis="columns", inplace=True)
assert len(top_h1) == len(undecaying) + len(decaying), "Classification dataframe length mismatch!"

In [ ]:
undecaying["classification"] = "undecaying"
decaying["classification"] = "decaying"
decaying.fillna(pd.NA, inplace=True)
result = pd.concat([undecaying, decaying], ignore_index=True)